In [12]:
import pandas as pd
import seaborn as sns
import datetime as dt
import io

In [6]:
import requests

r = requests.get("http://127.0.0.1:5000/forecasts?longitude=20.1&latitude=50&publication_date=2024-05-27")

In [17]:
pd.read_csv(io.StringIO(r.text), parse_dates=["time", "valid_time"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   time        64 non-null     datetime64[ns, UTC]
 1   valid_time  64 non-null     datetime64[ns, UTC]
 2   latitude    64 non-null     float64            
 3   longitude   64 non-null     float64            
 4   number      64 non-null     int64              
 5   u10         64 non-null     float64            
 6   v10         64 non-null     float64            
 7   tp          64 non-null     float64            
 8   tcc         64 non-null     float64            
 9   t2m         64 non-null     float64            
 10  prmsl       64 non-null     float64            
dtypes: datetime64[ns, UTC](2), float64(8), int64(1)
memory usage: 5.6 KB


In [18]:
len(str(115706033947377433202))

21

In [2]:
locations = pd.read_csv('app/locations.csv', index_col="id")

In [3]:
latitude = locations.loc[locations["location"].str.startswith("Warsaw"), "latitude"].iloc[0]
longitude = locations.loc[locations["location"].str.startswith("Warsaw"), "longitude"].iloc[0]

In [4]:
min_latitude = int(latitude)
max_latitude = int(latitude) + 1

min_longitude = int(longitude)
max_longitude = int(longitude) + 1

min_latitude, max_latitude, min_longitude, max_longitude

(52, 53, 21, 22)

In [5]:
latitude, longitude

(52.23, 21.0111)

In [6]:
import pandas_gbq

In [17]:
publication_date = dt.date(2024, 5, 28)
publication_date_str = publication_date.strftime("%Y-%m-%d")

pandas_gbq.read_gbq(
    f"""                    
SELECT 
  *
FROM `meteo_dataset.gefs` 
WHERE time = '{publication_date_str} 00:00:00 UTC'
  and latitude in ({min_latitude}, {max_latitude})
  and longitude in ({min_longitude}, {max_longitude})
""",
    progress_bar_type=None,
)

,time,valid_time,latitude,longitude,number,u10,v10,tp,tcc,t2m,prmsl
0,2024-05-28 00:00:00+00:00,2024-05-29 12:00:00+00:00,52.0,21.0,-1,0.96,0.93,2.2,44.0,296.000000,101281.765625
1,2024-05-28 00:00:00+00:00,2024-05-29 12:00:00+00:00,52.0,22.0,-1,-0.86,2.93,2.7,43.0,296.700012,101276.164062
2,2024-05-28 00:00:00+00:00,2024-05-30 12:00:00+00:00,52.0,22.0,-1,-2.07,2.20,1.8,72.0,296.000000,100762.296875
3,2024-05-28 00:00:00+00:00,2024-05-29 12:00:00+00:00,53.0,21.0,-1,0.93,1.85,3.8,46.0,295.500000,101290.765625
4,2024-05-28 00:00:00+00:00,2024-05-29 12:00:00+00:00,53.0,22.0,-1,-0.88,3.84,4.2,49.0,296.200012,101291.164062
...,...,...,...,...,...,...,...,...,...,...,...
59,2024-05-28 00:00:00+00:00,2024-06-07 12:00:00+00:00,53.0,22.0,-1,2.21,-0.34,0.9,42.0,293.299988,101474.039062
60,2024-05-28 00:00:00+00:00,2024-06-01 12:00:00+00:00,52.0,21.0,-1,2.41,2.50,1.3,64.0,295.690002,101048.109375
61,2024-05-28 00:00:00+00:00,2024-05-31 12:00:00+00:00,53.0,21.0,-1,0.14,1.83,1.3,53.0,296.989990,100717.148438
62,2024-05-28 00:00:00+00:00,2024-06-03 12:00:00+00:00,53.0,22.0,-1,2.16,1.10,1.3,61.0,295.100006,101310.085938


In [11]:
# SELECT 
#   *
# FROM `cloud-computing-2024-424511.meteo_dataset.gefs` 
# WHERE time = '2024-05-27 00:00:00 UTC'
#   and latitude in (52, 53)
#   and longitude in (21, 22)

In [15]:
df = pd.read_csv("app/modules/bquxjob_7625a88d_18fbeca86b6.csv", parse_dates=["time", "valid_time"])


In [19]:
df = pd.read_csv("bquxjob_7625a88d_18fbeca86b6.csv", parse_dates=["time", "valid_time"])
df = df.sort_values(by=["time", "valid_time"])

df["distance"] = 2 - (
    (df["latitude"] - latitude) ** 2 + (df["longitude"] - longitude) ** 2
)

distance_norm = (
    df.groupby(["time", "valid_time"])[["distance"]]
    .sum()
    .reset_index()
    .rename(columns={"distance": "distance_norm"})
)
df = df.merge(distance_norm, on=["time", "valid_time"])

df = (
    df.assign(
        distance=lambda x: x["distance"] / x["distance_norm"],
        u10=lambda x: x["u10"] * x["distance"],
        v10=lambda x: x["v10"] * x["distance"],
        tp=lambda x: x["tp"] * x["distance"],
        tcc=lambda x: x["tcc"] * x["distance"],
        t2m=lambda x: x["t2m"] * x["distance"],
        prmsl=lambda x: x["prmsl"] * x["distance"],
    )
    .groupby(["time", "valid_time"])[["u10", "v10", "tp", "tcc", "t2m", "prmsl"]]
    .sum()
    .reset_index()
    .assign(
        w=lambda x: (x["u10"] ** 2 + x["v10"] ** 2) ** 1 / 2,
        t2m=lambda x: x["t2m"] - 273.15,
        prmsl=lambda x: x["prmsl"] / 100,
    )
    .drop(columns=["u10", "v10"])
)

# tp - kg/m^2
# tcc - %
# t2m - K
# prmsl - Pa
# u10, v10 - m/s
df

,time,valid_time,tp,tcc,t2m,prmsl,w
0,2024-05-27 00:00:00+00:00,2024-05-27 12:00:00+00:00,0.000000,4.328844,24.614530,1021.315277,16.751403
1,2024-05-27 00:00:00+00:00,2024-05-28 12:00:00+00:00,0.115730,33.786330,24.393660,1016.336755,17.505193
2,2024-05-27 00:00:00+00:00,2024-05-29 12:00:00+00:00,2.402942,51.017876,22.812785,1013.015385,0.828742
3,2024-05-27 00:00:00+00:00,2024-05-30 12:00:00+00:00,2.018969,85.705753,21.159516,1007.350730,2.646030
4,2024-05-27 00:00:00+00:00,2024-05-31 12:00:00+00:00,1.577819,49.030578,23.963629,1007.313727,2.674650
5,2024-05-27 00:00:00+00:00,2024-06-01 12:00:00+00:00,1.618128,66.272742,22.171992,1011.329565,5.865431
6,2024-05-27 00:00:00+00:00,2024-06-02 12:00:00+00:00,0.843004,57.199650,23.104235,1013.924908,4.869388
7,2024-05-27 00:00:00+00:00,2024-06-03 12:00:00+00:00,1.804730,70.297876,21.383846,1013.791128,3.501204
8,2024-05-27 00:00:00+00:00,2024-06-04 12:00:00+00:00,1.095814,58.436640,21.784209,1014.856582,1.545223
9,2024-05-27 00:00:00+00:00,2024-06-05 12:00:00+00:00,0.863513,52.794247,22.318175,1014.528573,2.040617


In [20]:
df = df.drop(columns=["time"])
df["valid_time"] = df["valid_time"].dt.strftime("%d/%m")
df = df.set_index("valid_time")
df = df.rename_axis(None, axis=0)
df = df.loc[:, ["tcc", "t2m", "tp", "w", "prmsl"]]
gmap = df.copy()

def format_tcc(x):
    emoji = ""
    if 0 <= x.tcc < 30:
        emoji = "☀️"
    if 30 <= x.tcc < 60:
        emoji = "🌤️"
    if 60 <= x.tcc < 90:
        if x.tp > 0.5:
            emoji = "🌦️"
        else:
            emoji = "🌥️"
    if 80 <= x.tcc:
        if x.tp > 0.5:
            emoji = "🌧️"
        else:
            emoji = "☁️"
    return f"{emoji} {x.tcc/100:.0%}"

df["tcc"] = df.apply(format_tcc, axis=1)
df["tp"] = df["tp"].apply(lambda x: f"{x:.1f}" if x > 0.5 else "")
df["t2m"] = df["t2m"].apply(lambda x: f"{x:.0f}°")
df["w"] = df["w"].apply(lambda x: f"{x:.1f}")
df["prmsl"] = df["prmsl"].apply(lambda x: f"{x:.0f}")

column_mapper = {
    "tcc": "🌥️ Cloudiness [%]",
    "t2m": "🌡️ Temperature [°C]",
    "tp": "💧 Precipitation [mm]",
    "w": "💨 Wind [km/h]",
    "prmsl": "🌀 Pressure [hPa]"
}

df = df.rename(columns=column_mapper).T
gmap = gmap.rename(columns=column_mapper).T

In [22]:
df

,27/05,28/05,29/05,30/05,31/05,01/06,02/06,03/06,04/06,05/06,06/06,07/06,08/06,09/06,10/06,11/06
🌥️ Cloudiness [%],☀️ 4%,🌤️ 34%,🌤️ 51%,🌧️ 86%,🌤️ 49%,🌦️ 66%,🌤️ 57%,🌦️ 70%,🌤️ 58%,🌤️ 53%,🌤️ 45%,🌤️ 53%,🌦️ 61%,🌤️ 50%,🌤️ 56%,🌤️ 56%
🌡️ Temperature [°C],25°,24°,23°,21°,24°,22°,23°,21°,22°,22°,22°,21°,21°,22°,22°,22°
💧 Precipitation [mm],,,2.4,2.0,1.6,1.6,0.8,1.8,1.1,0.9,,0.8,0.6,,0.9,0.9
💨 Wind [km/h],16.8,17.5,0.8,2.6,2.7,5.9,4.9,3.5,1.5,2.0,2.9,1.5,0.9,1.1,1.3,0.3
🌀 Pressure [hPa],1021,1016,1013,1007,1007,1011,1014,1014,1015,1015,1014,1014,1013,1013,1013,1014


In [23]:
t2m_cmap = sns.diverging_palette(h_neg=255, h_pos=0, s=99, l=95, sep=46, as_cmap=True)
tp_cmap = sns.cubehelix_palette(start=2.2, rot=0.2, gamma=5, hue=1, light=0.9, dark=1, reverse=True, as_cmap=True)
w_cmap = sns.cubehelix_palette(start=2.3, rot=-1, gamma=4.8, hue=0, light=0.95, dark=1, reverse=True, as_cmap=True)
prmsl_cmap = sns.light_palette((280, 99, 85), as_cmap=True, input="husl")

df = (
    df.style
    .background_gradient(cmap=t2m_cmap, axis=1, subset=pd.IndexSlice["🌡️ Temperature [°C]":"🌡️ Temperature [°C]"], gmap=gmap.loc["🌡️ Temperature [°C]"])
    .background_gradient(cmap=tp_cmap, axis=1, subset=pd.IndexSlice["💧 Precipitation [mm]":"💧 Precipitation [mm]"], gmap=gmap.loc["💧 Precipitation [mm]"], vmin=0, vmax=10)
    .background_gradient(cmap=w_cmap, axis=1, subset=pd.IndexSlice["💨 Wind [km/h]":"💨 Wind [km/h]"], gmap=gmap.loc["💨 Wind [km/h]"], vmin=0, vmax=20)
    .background_gradient(cmap=prmsl_cmap, axis=1, subset=pd.IndexSlice["🌀 Pressure [hPa]":"🌀 Pressure [hPa]"], gmap=gmap.loc["🌀 Pressure [hPa]"], vmin=990, vmax=1030)
)
# df.to_html
df

,27/05,28/05,29/05,30/05,31/05,01/06,02/06,03/06,04/06,05/06,06/06,07/06,08/06,09/06,10/06,11/06
🌥️ Cloudiness [%],☀️ 4%,🌤️ 34%,🌤️ 51%,🌧️ 86%,🌤️ 49%,🌦️ 66%,🌤️ 57%,🌦️ 70%,🌤️ 58%,🌤️ 53%,🌤️ 45%,🌤️ 53%,🌦️ 61%,🌤️ 50%,🌤️ 56%,🌤️ 56%
🌡️ Temperature [°C],25°,24°,23°,21°,24°,22°,23°,21°,22°,22°,22°,21°,21°,22°,22°,22°
💧 Precipitation [mm],,,2.4,2.0,1.6,1.6,0.8,1.8,1.1,0.9,,0.8,0.6,,0.9,0.9
💨 Wind [km/h],16.8,17.5,0.8,2.6,2.7,5.9,4.9,3.5,1.5,2.0,2.9,1.5,0.9,1.1,1.3,0.3
🌀 Pressure [hPa],1021,1016,1013,1007,1007,1011,1014,1014,1015,1015,1014,1014,1013,1013,1013,1014
